In [ ]:
import dapla as dp
import pandas as pd

In [ ]:
# Se alle "mock"-datasettene i /felles/ på dapla
dp.show('/felles/mock')

In [ ]:
# Stapp datasettene som "objekter" inn i ett "variabelnavn"
comp = dp.read_pandas('/felles/mock_sysselsatte/companies_2019_30000')
pop = dp.read_pandas('/felles/mock_sysselsatte/population_2019_30000')

In [ ]:
#Print preview av ett datasett
comp

In [ ]:
# Og det andre
pop

### Døp om variabler innad i kolonner, er dette "omkoding"?

In [ ]:
# En dict av dicts, kolonnenavn: verdi - fra:til
omkodingskatalog = {
    'sex' : {
        'Menn' : 1,
        'Kvinner' : 2
    },
    'region' : {
        'Akershus (-2019)' : 'Akershus',
        'Hordaland (-2019)' : 'Hordaland',
        'Trøndelag - Trööndelage' : 'Trøndelag'
    }
}

# En av dictene i dicten
print(omkodingskatalog['sex'])
print(type(omkodingskatalog['sex']))

In [ ]:
pop.replace(omkodingskatalog)

In [ ]:
# Sjekk om det faktisk er omkodet, om replace() er "inplace"
pop

In [ ]:
comp.replace(omkodingskatalog)

### Opprett en ny kolonne med "grupperingsvariabler", avhengig av en "katalog" fra KLASS

Landsinndeling
https://www.ssb.no/klass/klassifikasjoner/106

KLASS-API
https://data.ssb.no/api/klass/v1/api-guide.html

In [ ]:
import json
import requests

# KLASS-query
url = 'http://data.ssb.no/api/klass/v1/correspondencetables/468'
headers = {'Accept': 'application/json'}
# Gjoer etterspoersel
r = requests.get(url, headers = headers)
# Av en eller annen grunn returnerer KLASS APIen json som bytestring...
# La oss decode...
cont = r.content.decode('utf8').replace("'", '"')
# Konverter til en liste vi kan bruke
cont = json.loads(cont)
cont = cont['correspondenceMaps']

In [ ]:
# Det vi har fått fra KLASS er nå en liste med dicts
cont

In [ ]:
# Loop over katalogen (så vi har tilgang på hver dict i listen)
for corr in cont:
    # En slags progress-rapport?
    print(corr['targetCode'], corr['sourceCode'])
    # Lag en ny kolonne for å holde på landsdel_koden, sett denne til sourceCode i katalogen, 
    # der region-koden matcher på targetCode.
    comp.loc[comp['region_code'] == corr['targetCode'], 'landsdel_kode' ] = corr['sourceCode']
    # Samme, bare for navnet på landsdelen
    comp.loc[comp['region_code'] == corr['targetCode'], 'landsdel_navn' ] = corr['sourceName']

In [ ]:
comp[['region', 'landsdel_navn']].sample(10)

In [ ]:
#Unike fylker som fortsatt mangler landsinndeling
pd.unique(comp[comp['landsdel_kode'].isna()]['region'])

In [ ]:
# Da må man enten gå gjennom å knytte disse gamle fylkene til landsdeler, eller finne flere korrespondansetabeller
fylk_kod = pd.unique(comp[comp['landsdel_kode'].isna()]['region_code'])
fylk_kod

<hr>

### Bonus, mer avansert kode: tildel de resterende fylkene landsdeler med "Excelredigering" i ipysheet

In [ ]:
import ipysheet as sheet
import numpy as np

In [ ]:
df_landsd = pd.DataFrame(fylk_kod)
df_landsd.columns = ['region_code']
df_landsd['region'] = ''
df_landsd['landsdel_kode'] = ''

In [ ]:
comp[comp['region_code'] == '02']['region'].iloc[0]

In [ ]:
for i, r in df_landsd.iterrows():
    print(r['region_code'])
    if r['region_code']:
        print(comp[comp['region_code'] == r['region_code']]['region'].iloc[0])
        r['region'] = comp[comp['region_code'] == r['region_code']]['region'].iloc[0]

In [ ]:
df_landsd

In [ ]:
last = ''
for corr in cont:
    kod = corr['sourceCode']
    nam = corr['sourceName']
    if last != kod:
        print(kod, ' : ', nam)
    last = kod

In [ ]:
landsdel_sheet = sheet.from_dataframe(df_landsd)
landsdel_sheet 

In [ ]:
df_edit = sheet.to_dataframe(landsdel_sheet)
df_edit

In [ ]:
for i, r in df_edit.iterrows():
    reg = r['region_code']
    lands = r['landsdel_kode']
    
    if lands:
        comp.loc[comp['region_code'] == reg, 'landsdel_kode' ] = lands
        
        # Finn landsdelnavn utifra landsdelkode
        for corr in cont:
            kod = corr['sourceCode']
            nam = corr['sourceName']
            if lands == kod:
                break
        else:
            raise KeyError(f'Finner ikke landsdel: {lands}')
                        
            
        
        comp.loc[comp['region_code'] == reg, 'landsdel_navn' ] = nam

In [ ]:
comp